This work is influenced from [this notebook.](https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/10_time_series_forecasting_in_tensorflow.ipynb)

In [1]:
#download the data
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv

--2023-10-06 10:18:41--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 178509 (174K) [text/plain]
Saving to: ‘BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv’

BTC_USD_2013-10-01_ 100%[===================>] 174.33K  --.-KB/s    in 0.01s   

2023-10-06 10:18:41 (14.9 MB/s) - ‘BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv’ saved [178509/178509]



In [2]:
#download the libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import random
from sklearn.preprocessing import MinMaxScaler

SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [3]:
df = pd.read_csv("/content/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv",
                 parse_dates=["Date"],
                 index_col=["Date"])
df.head()


,Currency,Closing Price (USD),24h Open (USD),24h High (USD),24h Low (USD)
Date,,,,,
2013-10-01,BTC,123.65499,124.30466,124.75166,122.56349
2013-10-02,BTC,125.45500,123.65499,125.75850,123.63383
2013-10-03,BTC,108.58483,125.45500,125.66566,83.32833
2013-10-04,BTC,118.67466,108.58483,118.67500,107.05816
2013-10-05,BTC,121.33866,118.67466,121.93633,118.00566


In [4]:
# Only want closing price for each day
bitcoin_prices = pd.DataFrame(df["Closing Price (USD)"]).rename(columns={"Closing Price (USD)": "Price"})
bitcoin_prices.head()

,Price
Date,
2013-10-01,123.65499
2013-10-02,125.45500
2013-10-03,108.58483
2013-10-04,118.67466
2013-10-05,121.33866


In [5]:
import numpy as np
# Block reward values
block_reward_1 = 50 # 3 January 2009 (2009-01-03) - this block reward isn't in our dataset (it starts from 01 October 2013)
block_reward_2 = 25 # 28 November 2012
block_reward_3 = 12.5 # 9 July 2016
block_reward_4 = 6.25 # 11 May 2020

# Block reward dates (datetime form of the above date stamps)
block_reward_2_datetime = np.datetime64("2012-11-28")
block_reward_3_datetime = np.datetime64("2016-07-09")
block_reward_4_datetime = np.datetime64("2020-05-11")

In [6]:
# Get date indexes for when to add in different block dates
block_reward_2_days = (block_reward_3_datetime - bitcoin_prices.index[0]).days
block_reward_3_days = (block_reward_4_datetime - bitcoin_prices.index[0]).days
block_reward_2_days, block_reward_3_days

(1012, 2414)

In [7]:
# Add block_reward column
bitcoin_prices_block = bitcoin_prices.copy()
bitcoin_prices_block["block_reward"] = None

# Set values of block_reward column (it's the last column hence -1 indexing on iloc)
bitcoin_prices_block.iloc[:block_reward_2_days, -1] = block_reward_2
bitcoin_prices_block.iloc[block_reward_2_days:block_reward_3_days, -1] = block_reward_3
bitcoin_prices_block.iloc[block_reward_3_days:, -1] = block_reward_4
bitcoin_prices_block.head()

,Price,block_reward
Date,,
2013-10-01,123.65499,25
2013-10-02,125.45500,25
2013-10-03,108.58483,25
2013-10-04,118.67466,25
2013-10-05,121.33866,25


We create multivariate time series data by using `shift()` method of pandas.

In [8]:
#set up the parameters
HORIZON = 1
WINDOW_SIZE = 7

#copy the dataframe to generate multivariete time series object
bitcoin_prices_windowed = bitcoin_prices_block.copy()
for i in range(WINDOW_SIZE):
  bitcoin_prices_windowed[f"Price+{i+1}"] = bitcoin_prices_windowed['Price'].shift(periods = i+1)

#control the process
bitcoin_prices_windowed.head()

,Price,block_reward,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7
Date,,,,,,,,,
2013-10-01,123.65499,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-02,125.45500,25,123.65499,NaN,NaN,NaN,NaN,NaN,NaN
2013-10-03,108.58483,25,125.45500,123.65499,NaN,NaN,NaN,NaN,NaN
2013-10-04,118.67466,25,108.58483,125.45500,123.65499,NaN,NaN,NaN,NaN
2013-10-05,121.33866,25,118.67466,108.58483,125.45500,123.65499,NaN,NaN,NaN


In [9]:
#drop na values
X = bitcoin_prices_windowed.dropna().drop('Price',axis = 1).astype(np.float32)
y = bitcoin_prices_windowed.dropna()['Price'].astype(np.float32)
X.head()

,block_reward,Price+1,Price+2,Price+3,Price+4,Price+5,Price+6,Price+7
Date,,,,,,,,
2013-10-08,25.0,121.794998,120.655327,121.338661,118.674660,108.584831,125.455002,123.654991
2013-10-09,25.0,123.032997,121.794998,120.655327,121.338661,118.674660,108.584831,125.455002
2013-10-10,25.0,124.049004,123.032997,121.794998,120.655327,121.338661,118.674660,108.584831
2013-10-11,25.0,125.961159,124.049004,123.032997,121.794998,120.655327,121.338661,118.674660
2013-10-12,25.0,125.279663,125.961159,124.049004,123.032997,121.794998,120.655327,121.338661


In [10]:
train_size = int(0.8*len(X))
X_train,y_train = X[:train_size],y[:train_size]
X_test,y_test = X[train_size:],y[train_size:]

**Trial 1:** Base model

In [11]:
#create model dict
models = {}
histories = {}
MODEL_NAME = 'model1'
HISTORY_NAME = 'history1'

models[MODEL_NAME] = tf.keras.Sequential([
    layers.Dense(128,input_shape = (8,),activation = 'relu'),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(HORIZON,activation = 'linear')
],name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = ['mae']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(X_train,y_train,
                                                 epochs = 100,
                                                 validation_data = (X_test,y_test),
                                                 callbacks = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath = f'./ModelCheckpoints/{MODEL_NAME}.ckpt',
                                                    monitor = 'val_loss',
                                                    save_best_only = True,
                                                    save_weights_only = True
                                                 ),
                                                 batch_size = 32,
                                                 verbose = 0)


In [12]:
best_model1 = tf.keras.models.clone_model(models[MODEL_NAME])

#load the best weight
best_model1.load_weights('./ModelCheckpoints/model1.ckpt')

best_model1.compile(optimizer = 'adam',
    loss = 'mse',
    metrics = ['mae'])

#evalutate the model
best_model1.evaluate(X_test,y_test)

18/18 [==============================] - 0s 3ms/step - loss: 1162979.1250 - mae: 569.2101


[1162979.125, 569.2101440429688]

**Trial 2:** Change the loss to `huber`.

In [13]:
MODEL_NAME = 'model2'
HISTORY_NAME = 'history2'
LOSS = 'huber'
models[MODEL_NAME] = tf.keras.Sequential([
    layers.Dense(128,input_shape = (8,),activation = 'relu'),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(HORIZON,activation = 'linear')
],name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(X_train,y_train,
                                                 epochs = 100,
                                                 validation_data = (X_test,y_test),
                                                 callbacks = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath = f'./ModelCheckpoints/{MODEL_NAME}.ckpt',
                                                    monitor = 'val_loss',
                                                    save_best_only = True,
                                                    save_weights_only = True
                                                 ),
                                                 batch_size = 32,
                                                 verbose = 0)

In [14]:
best_model2 = tf.keras.models.clone_model(models[MODEL_NAME])

#load the best weight
best_model2.load_weights('./ModelCheckpoints/model2.ckpt')

best_model2.compile(optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae'])

#evalutate the model
best_model2.evaluate(X_test,y_test)

18/18 [==============================] - 0s 2ms/step - loss: 562.3800 - mae: 562.8796


[562.3800048828125, 562.879638671875]

**Trial 3:** Change loss to `mae`.

In [15]:
MODEL_NAME = 'model3'
HISTORY_NAME = 'history3'
LOSS = 'mae'
ACTIVATION = 'relu'

models[MODEL_NAME] = tf.keras.Sequential([
   layers.Dense(128,activation = ACTIVATION),
    layers.Dense(64,activation = ACTIVATION),
    layers.Dense(HORIZON,activation = 'linear')
],name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(X_train,y_train,
                                                 epochs = 100,
                                                 validation_data = (X_test,y_test),
                                                 callbacks = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath = f'./ModelCheckpoints/{MODEL_NAME}.ckpt',
                                                    monitor = 'val_loss',
                                                    save_best_only = True,
                                                    save_weights_only = True
                                                 ),
                                                 batch_size = 32,
                                                 verbose = 0)

In [16]:
best_model3 = tf.keras.models.clone_model(models[MODEL_NAME])

#load the best weight
best_model3.load_weights('./ModelCheckpoints/model3.ckpt')

best_model3.compile(optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae'])

#evalutate the model
best_model3.evaluate(X_test,y_test)

18/18 [==============================] - 0s 3ms/step - loss: 564.5666 - mae: 564.5666


[564.5665893554688, 564.5665893554688]

**Trial 4:** Change loss to `mape`.

In [17]:
MODEL_NAME = 'model4'
HISTORY_NAME = 'history4'
LOSS = 'mape'
ACTIVATION = 'relu'

models[MODEL_NAME] = tf.keras.Sequential([
   layers.Dense(128,activation = ACTIVATION),
    layers.Dense(64,activation = ACTIVATION),
    layers.Dense(HORIZON,activation = 'linear')
],name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(X_train,y_train,
                                                 epochs = 100,
                                                 validation_data = (X_test,y_test),
                                                 callbacks = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath = f'./ModelCheckpoints/{MODEL_NAME}.ckpt',
                                                    monitor = 'val_loss',
                                                    save_best_only = True,
                                                    save_weights_only = True
                                                 ),
                                                 batch_size = 32,
                                                 verbose = 0)

In [18]:
best_model4 = tf.keras.models.clone_model(models[MODEL_NAME])

#load the best weight
best_model4.load_weights('./ModelCheckpoints/model4.ckpt')

best_model4.compile(optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae'])

#evalutate the model
best_model4.evaluate(X_test,y_test)

18/18 [==============================] - 0s 3ms/step - loss: 2.5216 - mae: 562.2441


[2.521616220474243, 562.244140625]

**Trial 5:** Try to normalize.

In [19]:
minmax = MinMaxScaler()
X_train_scaled = minmax.fit_transform(X_train)

#transform the test data
X_test_scaled = minmax.transform(X_test)

In [20]:
MODEL_NAME = 'model5'
HISTORY_NAME = 'history5'
LOSS = 'huber'
models[MODEL_NAME] = tf.keras.Sequential([
    layers.Dense(128,input_shape = (8,),activation = 'relu'),
    layers.Dense(64,activation = 'relu'),
    layers.Dense(HORIZON,activation = 'linear')
],name = MODEL_NAME)

#compile the model
models[MODEL_NAME].compile(
    optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae']
)

#fit the model
histories[HISTORY_NAME] = models[MODEL_NAME].fit(X_train_scaled,y_train,
                                                 epochs = 100,
                                                 validation_data = (X_test_scaled,y_test),
                                                 callbacks = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath = f'./ModelCheckpoints/{MODEL_NAME}.ckpt',
                                                    monitor = 'val_loss',
                                                    save_best_only = True,
                                                    save_weights_only = True
                                                 ),
                                                 batch_size = 32,
                                                 verbose = 0)

In [22]:
best_model5 = tf.keras.models.clone_model(models[MODEL_NAME])

#load the best weight
best_model5.load_weights('./ModelCheckpoints/model5.ckpt')

best_model5.compile(optimizer = 'adam',
    loss = LOSS,
    metrics = ['mae'])

#evalutate the model
best_model5.evaluate(X_test_scaled,y_test)

18/18 [==============================] - 0s 3ms/step - loss: 893.1815 - mae: 893.6815


[893.1815185546875, 893.6815185546875]